## Reduced Rank Regression

In [ ]:
import os
import pickle as pkl
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from utils.helpers import *

# autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
# Load the data
current_dir = os.getcwd()
print("Loading the data...")
data_path = os.path.join(current_dir, 'data')
data = load_data(data_path, 'ssp585_time_series.pkl')
print("Data loaded successfully!")

In [ ]:
filtered_data = filter_data(data)

In [ ]:
# Print the shape of the first run in the first model of filtered_data
first_model = list(filtered_data.keys())[0]
first_run = list(filtered_data[first_model].keys())[0]
print(f"Shape of the first run in the first model: {filtered_data[first_model][first_run].shape}")

In [ ]:
# Create a NaN mask to remove any cells which have 1 >= NaNs

nan_mask = create_nan_mask(filtered_data)

In [ ]:
# Apply the NaN mask to every single run (modifications done in place for memory efficiency)

masked_data = mask_out_nans(filtered_data, nan_mask)

In [ ]:
# Print the shape of the first run in the first model of masked_data
first_model = list(masked_data.keys())[0]
first_run = list(masked_data[first_model].keys())[0]
print(f"Shape of the first run in the first model: {masked_data[first_model][first_run].shape}")

## The NaN mask has been applied and only the models with more than 4 runs have been kept

In [ ]:
print(f"There are exactly {len(masked_data.keys())} models of the {len(data.keys())} that remain.")

In [ ]:
reshaped_data = reshape_data(masked_data)

In [ ]:
example_model = 'CanESM5-1'
example_model_data = reshaped_data[example_model]
print(f"Selected model: {example_model}, which has {len(example_model_data)} runs.")

In [ ]:
first_example_model_run = example_model_data[list(example_model_data.keys())[0]]
first_example_model_run.shape # Shape: (Time x (Latitude x Longitude))

### Calculate the forced response for each model, assuming that it is the mean of all the runs

In [ ]:
data_forced_responses = add_forced_response(reshaped_data) # The forced response is not correctly normalized yet (this is done by the normalize function below)

In [ ]:
# Print the shape of a random run from a random model

random_model = random.choice(list(data_forced_responses.keys()))
random_run = random.choice(list(data_forced_responses[random_model].keys()))
print(f"Shape of the random run: {data_forced_responses[random_model][random_run].shape}")

In [ ]:
# Reshape the data to the original grid shape
data_grid = data_forced_responses[random_model][random_run][120].reshape(72, 144)

# Plot the data using plt.imshow
plt.figure(figsize=(10, 6))
plt.imshow(data_grid, cmap='viridis')
plt.colorbar(label='Value')
plt.title(f'Data for Model: {random_model}, Run: {random_run}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

## At this point, the data still contains NaNs, corresponding to the continents

In [ ]:
data_without_nans = remove_nans_from_grid(data_forced_responses, nan_mask)

In [ ]:
# Print the shape of a random run from a random model (same as above)
print(f"The shape of the data without NaNs: {data_without_nans[random_model][random_run].shape}")

## Now the NaNs have been completely removed from the data

### The data is now in the shape : Time x # of grid cells without NaNs

In [ ]:
# Select one of the models randomly for testing and the rest for training according to the leave-one-out strategy
test_model = random.choice(list(data_without_nans.keys()))
train_models = [model for model in data_without_nans.keys() if model != test_model]

# Create the training and testing datasets
train_data = {model: data_without_nans[model] for model in train_models}
test_data = {test_model: data_without_nans[test_model]}

print(f"Training models: {train_models}")
print(f"Testing model: {test_model}")

In [ ]:
# Normalize the data
normalized_train_data, normalized_test_data, training_statistics, testing_statistics = normalize_data(train_data, test_data)

## Now that the data has been normalized (and NaN values removed), it's time to do the Reduced Rank Regression (RRR)

In [ ]:
# First pool the data together
X_full, Y_full = pool_data(normalized_train_data)

In [ ]:
X_full.shape, Y_full.shape # Shape is correct

In [ ]:
rank = 10 # Used as a starting point
lambda_ = 0.01 # This will be optimized later
Brr, B_ols = reduced_rank_regression(X_full, Y_full, rank, lambda_)

In [ ]:
# print mean and std of X_full and Y_full

print(f"Mean of X_full: {np.mean(X_full)}")
print(f"Std of X_full: {np.std(X_full)}")
print(f"Mean of Y_full: {np.mean(Y_full)}")
print(f"Std of Y_full: {np.std(Y_full)}")

In [ ]:
Brr.shape, B_ols.shape

In [ ]:
# Test a prediction on the test set for a single run

# Select a random run from the test set
test_model = list(normalized_test_data.keys())[0]
test_runs = [run for run in normalized_test_data[test_model].keys() if run != 'forced_response']
test_run = normalized_test_data[test_model][random.choice(test_runs)]
ground_truth = normalized_test_data[test_model]['forced_response']

In [ ]:
test_run.shape, ground_truth.shape

In [ ]:
# make the prediction

prediction = test_run @ B_ols
prediction.shape

In [ ]:
# Add the NaN values back to the correct spots
predictions = True
prediction = readd_nans_to_grid(prediction, nan_mask, predictions)
test_run = readd_nans_to_grid(test_run, nan_mask, predictions)
ground_truth = readd_nans_to_grid(ground_truth, nan_mask, predictions)

In [ ]:
prediction.shape, test_run.shape, ground_truth.shape

In [ ]:
prediction.shape, test_run.shape, ground_truth.shape

In [ ]:
# Select a random grid spot
valid_grid_spots = np.argwhere(~nan_mask)
random_grid_spot = valid_grid_spots[random.randint(0, len(valid_grid_spots))]
print(f"Displaying the forced response and runs for grid spot: {random_grid_spot}")

# Data for each run (minus the forced response)
input_data = test_run.reshape(-1, 72, 144)[:, random_grid_spot[0], random_grid_spot[1]]
# Data for the forced response
forced_response_data = ground_truth.reshape(-1, 72, 144)[:, random_grid_spot[0], random_grid_spot[1]]
# Data for the prediction
predicted_data = prediction.reshape(-1, 72, 144)[:, random_grid_spot[0], random_grid_spot[1]]

# Plot the time series data for all runs, the forced response, and the prediction
plt.figure(figsize=(10, 6))
plt.plot(input_data, color='blue', alpha=0.5, label='Test Run')
plt.plot(forced_response_data, color='red', label='Forced Response', linewidth=2)
plt.plot(predicted_data, color='green', label='Prediction', linewidth=2, linestyle='--')
plt.title(f'Time Evolution at Grid Spot ({random_grid_spot[0]}, {random_grid_spot[1]}) for Model: {test_model}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
print(f"The shape of the data is: {X_full.shape}")
print(f"The shape of the target is: {Y_full.shape}")
print(f"The maximum, minimum, average and std for the input data are: {np.max(X_full)}, {np.min(X_full)}, {np.mean(X_full)}, {np.std(X_full)}")
print(f"The maximum, minimum, average and std for the target data are: {np.max(Y_full)}, {np.min(Y_full)}, {np.mean(Y_full)}, {np.std(Y_full)}")

In [ ]:
# Plot a random timestamp for the target and prediction

random_timestamp = random.randint(0, prediction.shape[0])
print(f"Random timestamp: {random_timestamp}")

# Data for the target
# Reshape the data to the original grid shape
test_run_grid = test_run[random_timestamp, :].reshape(72, 144)
ground_truth_grid = ground_truth[random_timestamp, :].reshape(72, 144)
prediction_grid = prediction[random_timestamp, :].reshape(72, 144)

# Plot the data using plt.imshow
plt.figure(figsize=(18, 6))

# Plot the test run
plt.subplot(1, 3, 1)
plt.imshow(test_run_grid, cmap='viridis')
plt.colorbar(label='Value')
plt.title(f'Test Run input at Timestamp {random_timestamp}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Plot the ground truth (forced response)
plt.subplot(1, 3, 2)
plt.imshow(ground_truth_grid, cmap='viridis')
plt.colorbar(label='Value')
plt.title(f'Ground Truth at Timestamp {random_timestamp}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Plot the prediction
plt.subplot(1, 3, 3)
plt.imshow(prediction_grid, cmap='viridis')
plt.colorbar(label='Value')
plt.title(f'Prediction at Timestamp {random_timestamp}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()